In [3]:
import pandas as pd
import numpy as np

# Đọc dữ liệu từ file Excel
df = pd.read_excel('/kaggle/input/team2-dataset/team2_dataset/chap7.xlsx')

# Chọn các đặc trưng để tạo thành mảng X
features = ['Age', 'Income (1000s)', 'Cards have']
X = df[features].values

# Chọn cột 'Response' để tạo thành mảng Y
Y = df['Response'].values

# Chuyển đổi X và Y thành mảng numpy
X = np.array(X)
Y = np.array(Y)

# Tạo điểm dữ liệu mới
new_point = np.array([27, 155, 5])


# Định nghĩa lớp cho thuật toán KNN
class K_Nearest_Neighbors:
    
    @staticmethod
    def normalize_data(X, new_point):
        """
        Chuẩn hóa dữ liệu bằng công thức: normalize = (100 / (max - min)) * (value - min)
        """
        X_min = X.min(axis=0)
        X_max = X.max(axis=0)
        X_scaled = (100 / (X_max - X_min)) * (X - X_min)
        new_point_scaled = (100 / (X_max - X_min)) * (new_point - X_min)
        return X_scaled, new_point_scaled

    @staticmethod
    def euclidean(a, b):
        return np.sqrt(np.sum((a - b) ** 2))

    @staticmethod
    def compute_distances(X, new_point):
        distances = np.zeros(X.shape[0])
        for i in range(X.shape[0]):
            distances[i] = K_Nearest_Neighbors.euclidean(X[i], new_point)
        return distances

    @staticmethod
    def distance_K(X, k, new_point):
        distances = K_Nearest_Neighbors.compute_distances(X, new_point)
        nearest_neighbors = np.argsort(distances)[:k]
        return nearest_neighbors, None

    @staticmethod
    def weighted_distance_K(X, k, new_point, b):
        """
        Tính khoảng cách từ new_point đến từng láng giềng (cột distance)
        Tính trọng số nghịch đảo khoảng cách (cột Reciprocal)
        Chuẩn hóa trọng số nghịch đảo (cột weight)
        """
        distances = K_Nearest_Neighbors.compute_distances(X, new_point)
        sorted_indices = np.argsort(distances)
        nearest_neighbors = sorted_indices[:k]
                
        # Tính trọng số nghịch đảo khoảng cách
        reciprocal = 1 / (1 + b * distances[nearest_neighbors])
        
        # Chuẩn hóa trọng số nghịch đảo
        weights = reciprocal / np.sum(reciprocal)
        
        return nearest_neighbors, weights
    
    @staticmethod
    def predict_response(nearest_neighbors_responses):
        response_counts = {}
        for response in nearest_neighbors_responses:
            if response in response_counts:
                response_counts[response] += 1
            else:
                response_counts[response] = 1

        max_counts = -1
        predicted_response = None
        for response, count in response_counts.items():
            if count > max_counts:
                max_counts = count
                predicted_response = response
        return predicted_response

    @staticmethod
    def predict_weight_response(nearest_neighbors_responses, weights):
        total_weight = np.sum(weights)
        total_weight_response_1 = 0.0
        
        for response, weight in zip(nearest_neighbors_responses, weights):
            if response == 1:
                total_weight_response_1 += weight
        
        # Tính xác suất chấp nhận (Probability)
        if total_weight > 0:
            probability = total_weight_response_1 / total_weight
        else:
            probability = 0.0
        
        predicted_response = 1 if probability >= 0.5 else 0
        return predicted_response, probability

    @staticmethod
    def KNN(X, Y, new_point, k_run, weighted=True, normalize_data=True, b=1):
        for k in k_run:
            if weighted and normalize_data:
                X_scaled, new_point_scaled = K_Nearest_Neighbors.normalize_data(X, new_point)
                nearest_neighbors, weights = K_Nearest_Neighbors.weighted_distance_K(X_scaled, k, new_point_scaled, b)
                nearest_neighbors_responses = Y[nearest_neighbors]
                predict, probability = K_Nearest_Neighbors.predict_weight_response(nearest_neighbors_responses, weights)
                print(f'Predicted response for K={k} (Weighted=True, normalize_data=True): {predict}, Probability={probability:.3f}')

            elif weighted and not normalize_data:
                nearest_neighbors, weights = K_Nearest_Neighbors.weighted_distance_K(X, k, new_point, b)
                nearest_neighbors_responses = Y[nearest_neighbors]
                predict, probability = K_Nearest_Neighbors.predict_weight_response(nearest_neighbors_responses, weights)
                print(f'Predicted response for K={k} (Weighted=True, normalize_data=False): {predict}, Probability={probability:.3f}')
            else:
                nearest_neighbors, _ = K_Nearest_Neighbors.distance_K(X, k, new_point)
                nearest_neighbors_responses = Y[nearest_neighbors]
                predict = K_Nearest_Neighbors.predict_response(nearest_neighbors_responses)
                print(f'Predicted response for K={k} (Weighted=False): {predict}')
                
# Xác định các giá trị K để chạy thuật toán
k_run = [1, 3, 5, 7, 9]

# Thực hiện dự đoán nhãn cho điểm dữ liệu mới với nhiều giá trị của K, có trọng số
print("\nKNN có trọng số và chuẩn hóa:")
K_Nearest_Neighbors.KNN(X, Y, new_point, k_run, weighted=True, normalize_data=True, b=1)

# Thực hiện dự đoán nhãn cho điểm dữ liệu mới với nhiều giá trị của K, có trọng số
print("\nKNN có trọng số không chuẩn hóa:")
K_Nearest_Neighbors.KNN(X, Y, new_point, k_run, weighted=True, normalize_data=False, b=1)

# Thực hiện dự đoán nhãn cho điểm dữ liệu mới với nhiều giá trị của K, không trọng số
print("\nKNN không trọng số:")
K_Nearest_Neighbors.KNN(X, Y, new_point, k_run, weighted=False, normalize_data=False, b=1)



KNN có trọng số và chuẩn hóa:
Predicted response for K=1 (Weighted=True, normalize_data=True): 0, Probability=0.000
Predicted response for K=3 (Weighted=True, normalize_data=True): 0, Probability=0.201
Predicted response for K=5 (Weighted=True, normalize_data=True): 0, Probability=0.228
Predicted response for K=7 (Weighted=True, normalize_data=True): 0, Probability=0.213
Predicted response for K=9 (Weighted=True, normalize_data=True): 0, Probability=0.230

KNN có trọng số không chuẩn hóa:
Predicted response for K=1 (Weighted=True, normalize_data=False): 0, Probability=0.000
Predicted response for K=3 (Weighted=True, normalize_data=False): 0, Probability=0.468
Predicted response for K=5 (Weighted=True, normalize_data=False): 0, Probability=0.488
Predicted response for K=7 (Weighted=True, normalize_data=False): 0, Probability=0.467
Predicted response for K=9 (Weighted=True, normalize_data=False): 0, Probability=0.469

KNN không trọng số:
Predicted response for K=1 (Weighted=False): 0
Pr